In [68]:
import sys
import os

# Ajouter le chemin du répertoire de la bibliothèque
sys.path.append(os.path.abspath("C:/Users/LENOVO/Desktop/EDS/Programmation/project/surf_scrap"))

# importer la bibliothèque
from meteo_surf_scrap import extract_meteo_surf

#csv_file = extract_meteo_surf('https://www.surf-report.com/meteo-surf/lacanau-s1043.html')
#print(csv_file)

In [70]:
def extract_meteo_surf(url) :
    
    """
    This function takes a  url an user_agent, scrapes surf forecast data from a given url page and stores 
    data points in a csv_file
    
    - output : pandas dataframe contaning the data extracted
    """
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Initialize empty lists to hold the data needed :
    dates = []
    hours = []
    vagues = []
    wind_speeds = []
    wind_directions = []

    # Looping through each forecast-tab block (each block represents one day)
    forecast_tabs = soup.find_all('div', class_='forecast-tab')
    for tab in forecast_tabs:
        # Extracting the date for this tab
        date_div = tab.find('div', class_='title')
        forecast_date = date_div.find('b').text.strip() if date_div else 'N/A'

        # Extracting the hours, waves, wind speeds, and wind directions for this date
        time_divs = tab.find_all('div', class_='cell date with-border')
        for time_div in time_divs:
            hour = time_div.text.strip()
            hours.append(hour)
            dates.append(forecast_date)  # Repeat the date for each hour

        # Extracting waves (vagues)
        for wave_div in tab.find_all('div', class_='cell large waves with-border'):
            vague = wave_div.text.strip()
            vagues.append(vague)

        # Extracting wind speeds
        for wind_div in tab.select('div[class^="wind wind-color-"]'):
            wind_speed = wind_div.find('span').text.strip()
            wind_speeds.append(wind_speed)

        # Extracting wind directions
        for wind_dir_div in tab.find_all('div', class_='wind img'):
            wind_direction = wind_dir_div.find('img')['alt'] if wind_dir_div else 'N/A'
            wind_directions.append(wind_direction)

    # Creating a DataFrame with the extracted data
    df = pd.DataFrame({
        'Date': dates,
        'Hour': hours,
        'Vagues': vagues,
        'Wind Speed (km/h)': wind_speeds,
        'Wind Direction': wind_directions
    })

    df.to_csv('output.csv', index=False)
    return df

In [74]:
import argparse
def main():
    # Create the parser
    parser = argparse.ArgumentParser(description='Extract surf forecast data from a given URL.')

    # Add the URL argument
    parser.add_argument('--input_url', type=str, help='The URL of the surf forecast page to scrape.')

    # Parse the arguments
    args = vars(parser.parse_args())
    extract_meteo_surf(url=args["input_url"])
    
    # Display a message indicating successful completion
    print("Data extracted and saved to output.csv")
    print(df.head())  # Optionally print the first few rows of the DataFrame

if __name__ == "__main__":
    # Call the main function with the provided URL
    main()

usage: ipykernel_launcher.py [-h] [--input_url INPUT_URL]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\LENOVO\AppData\Roaming\jupyter\runtime\kernel-871f0796-264c-43f3-8759-4c8b9ed4eff8.json


SystemExit: 2

C:\Users\LENOVO\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [76]:
!python run_surf_scrap_library.py --help

usage: run_surf_scrap_library.py [-h] [--input_url INPUT_URL]

Extract surf forecast data from a given URL.

options:
  -h, --help            show this help message and exit
  --input_url INPUT_URL
                        The URL of the surf forecast page to scrape.


In [78]:
!python run_surf_scrap_library.py --input_url "https://www.surf-report.com/meteo-surf/lacanau-s1043.html"

                 Date   Hour  ... Wind Speed (km/h)                Wind Direction
0  Samedi 16 Novembre  01:00  ...                19          Orientation vent Est
1  Samedi 16 Novembre  04:00  ...                23  Orientation vent Est Sud Est
2  Samedi 16 Novembre  07:00  ...                20  Orientation vent Est Sud Est
3  Samedi 16 Novembre  10:00  ...                14  Orientation vent Est Sud Est
4  Samedi 16 Novembre  13:00  ...                 4      Orientation vent Sud Est

[5 rows x 5 columns]
Data extracted and saved to output.csv
